In [2]:
### Inference on a single image
import os
import json
import torch
import torch.nn as nn
from torchvision.models import EfficientNet_V2_S_Weights, efficientnet_v2_s
from PIL import Image

# --- Configuration ---
MODEL_PATH = 'models/efficientnetv2s/best_model.pth'
# IMAGE_PATH = r'dataset\images\test\Hummus\18.jpg'  # <<<--- Change this to your image file
IMAGE_PATH = r"dataset\images\test\Hummus\65570.jpg"
MAPPING_JSON_PATH = 'class_mapping.json'
NUM_CLASSES = 64

# --- Device Setup ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# --- Load Class Mapping ---
print(f"Loading class mapping from: {MAPPING_JSON_PATH}")
try:
    with open(MAPPING_JSON_PATH, 'r') as f:
        class_to_idx = json.load(f)
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    if len(idx_to_class) != NUM_CLASSES:
        print(f"Warning: Found {len(idx_to_class)} classes, expected {NUM_CLASSES}")
except Exception as e:
    raise RuntimeError(f"Failed to load class mapping: {e}")

# --- Load Model ---
print("Loading model...")
weights = EfficientNet_V2_S_Weights.IMAGENET1K_V1
model = efficientnet_v2_s(weights=weights)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Sequential(
    nn.Dropout(p=0.37, inplace=True), # Keep dropout
    nn.Linear(num_ftrs, NUM_CLASSES),
)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# --- Preprocessing ---
preprocess = weights.transforms()

# --- Prediction Function ---
def predict_image(image_path):
    try:
        img = Image.open(image_path).convert('RGB')
        input_tensor = preprocess(img).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            output = model(input_tensor)
            conf, idx = torch.max(torch.softmax(output, dim=1), 1)
        return idx_to_class.get(idx.item(), "Unknown"), conf.item()
    except Exception as e:
        print(f"Prediction error: {e}")
        return None, None

# --- Inference ---
if not os.path.exists(IMAGE_PATH):
    print(f"Image path does not exist: {IMAGE_PATH}")
else:
    print(f"Predicting image: {IMAGE_PATH}")
    label, confidence = predict_image(IMAGE_PATH)
    if label:
        print(f"-> Predicted Class: {label}")
        print(f"-> Confidence: {confidence:.4f}")

Using device: cuda
Loading class mapping from: class_mapping.json
Loading model...


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_24848\3271592363.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH, map_location=

Predicting image: dataset\images\test\Hummus\65570.jpg
-> Predicted Class: Cucumber
-> Confidence: 0.3880


In [2]:
IMAGE_PATH = r"dataset\images\train"
import os
# ...existing code...

import shutil
import random

def copy_sample_images(src_root, dst_root, samples_per_folder=10, image_exts=('.jpg', '.jpeg', '.png', '.bmp', '.webp')):
    """
    Copies up to `samples_per_folder` random images from each subfolder in `src_root`
    to a corresponding subfolder in `dst_root`.
    """
    if not os.path.exists(dst_root):
        os.makedirs(dst_root)
    for subdir, dirs, files in os.walk(src_root):
        # Only process leaf directories (those containing images)
        image_files = [f for f in files if f.lower().endswith(image_exts)]
        if image_files:
            rel_path = os.path.relpath(subdir, src_root)
            dst_subdir = os.path.join(dst_root, rel_path)
            os.makedirs(dst_subdir, exist_ok=True)
            sample_files = random.sample(image_files, min(samples_per_folder, len(image_files)))
            for fname in sample_files:
                src_file = os.path.join(subdir, fname)
                dst_file = os.path.join(dst_subdir, fname)
                shutil.copy2(src_file, dst_file)
                print(f"Copied: {src_file} -> {dst_file}")

# Example usage:
src_images_root = r"dataset\images\train"         # Source root directory
dst_images_root = r"dataset\images\test_sample_for_client"  # Destination root directory

copy_sample_images(src_images_root, dst_images_root, samples_per_folder=10)

Copied: dataset\images\train\Almonds\image_5.jpg -> dataset\images\test_sample_for_client\Almonds\image_5.jpg
Copied: dataset\images\train\Almonds\image_107.jpg -> dataset\images\test_sample_for_client\Almonds\image_107.jpg
Copied: dataset\images\train\Almonds\image_55.png -> dataset\images\test_sample_for_client\Almonds\image_55.png
Copied: dataset\images\train\Almonds\image_3.jpg -> dataset\images\test_sample_for_client\Almonds\image_3.jpg
Copied: dataset\images\train\Almonds\image_36.png -> dataset\images\test_sample_for_client\Almonds\image_36.png
Copied: dataset\images\train\Almonds\image_32.jpg -> dataset\images\test_sample_for_client\Almonds\image_32.jpg
Copied: dataset\images\train\Almonds\image_87.jpg -> dataset\images\test_sample_for_client\Almonds\image_87.jpg
Copied: dataset\images\train\Almonds\image_26.png -> dataset\images\test_sample_for_client\Almonds\image_26.png
Copied: dataset\images\train\Almonds\image_59.png -> dataset\images\test_sample_for_client\Almonds\image_5

#### Yolov8 inference

In [7]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import os
import glob

# Load the YOLOv8x model pretrained on Open Images V7
# model = YOLO(r"models\yolov8m-oiv7\weights\best.pt")
model = YOLO(r"models\yolov8m-oiv7\train4\weights\best.pt")

# Input and output directories
input_dir = "dataset/images/test/Dates_with_tahini"
output_dir = "outputs/"
os.makedirs(output_dir, exist_ok=True)

# Supported image formats
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.webp']

# Get all image paths
image_paths = []
for ext in image_extensions:
    image_paths.extend(glob.glob(os.path.join(input_dir, ext)))

print(f"Found {len(image_paths)} image(s) to process.")

# Process each image
for img_path in image_paths:
    print(f"\nProcessing: {img_path}")
    
    # Run prediction
    results = model.predict(source=img_path, save=False, show=False)

    for i, r in enumerate(results):
        # print("\n--- Detection Summary ---")
        # print(f"Boxes: {r.boxes.xyxy}")
        # print(f"Classes: {r.boxes.cls}")
        # print(f"Scores: {r.boxes.conf}")
        # print(f"Class names: {[r.names[int(i)] for i in r.boxes.cls]}")

        # Generate annotated image
        annotated_image = r.plot()

        # Construct output file path
        base_name = os.path.basename(img_path)
        output_path = os.path.join(output_dir, f"annotated_{base_name}")

        # Save image (convert to BGR for OpenCV)
        cv2.imwrite(output_path, annotated_image)
        print(f"Saved annotated image to: {output_path}")

        # Optional: Display the image
        # plt.imshow(annotated_image)
        # plt.axis('off')
        # plt.title(f"Detections for {base_name}")
        # plt.show()

Found 6 image(s) to process.

Processing: dataset/images/test/Dates_with_tahini\test_69.jpg

image 1/1 c:\Work\Garbage_classification\food-recognition\food-recognition\dataset\images\test\Dates_with_tahini\test_69.jpg: 640x448 2 Foods, 77.3ms
Speed: 3.0ms preprocess, 77.3ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 448)
Saved annotated image to: outputs/annotated_test_69.jpg

Processing: dataset/images/test/Dates_with_tahini\train_61.jpg

image 1/1 c:\Work\Garbage_classification\food-recognition\food-recognition\dataset\images\test\Dates_with_tahini\train_61.jpg: 448x640 1 Baked goods, 7 Cookies, 2 Desserts, 25.5ms
Speed: 2.5ms preprocess, 25.5ms inference, 1.3ms postprocess per image at shape (1, 3, 448, 640)
Saved annotated image to: outputs/annotated_train_61.jpg

Processing: dataset/images/test/Dates_with_tahini\train_75.jpg

image 1/1 c:\Work\Garbage_classification\food-recognition\food-recognition\dataset\images\test\Dates_with_tahini\train_75.jpg: 640x448 3 Bak

In [1]:
import json

with open('utils/local_nutrition_db.json', 'r') as f:
    local_nutrition_db = json.load(f)

In [4]:
sorted(list(local_nutrition_db.keys()))

['apple',
 'artichoke',
 'bagel',
 'balaleet',
 'bamya',
 'banana',
 'basbousa',
 'beer',
 'bell pepper',
 'bread',
 'broccoli',
 'burger',
 'burrito',
 'cabbage',
 'cake',
 'candy',
 'cantaloupe',
 'carrot',
 'cheese',
 'cocktail',
 'common fig',
 'cookie',
 'crab',
 'cream',
 'croissant',
 'cucumber',
 'dates_with_tahini',
 'fattoush',
 'fish',
 'french fries',
 'fries',
 'garden asparagus',
 'gers_ogaily',
 'grape',
 'grapefruit',
 'guacamole',
 'hamburger',
 'harees',
 'hot dog',
 'hummus',
 'ice cream',
 'jireesh',
 'kebab',
 'khabeesa',
 'kubba',
 'laban_drink',
 'labneh',
 'lemon',
 'lentil_soup',
 'lobster',
 'luqaimat',
 'majboos_dajaj',
 'mallooba(maqluba)',
 'mango',
 'modas_rice',
 'molokhia',
 'muffin',
 'muhammara',
 'murabyan',
 'musakhan_chicken',
 'mushroom',
 'om_ali',
 'orange',
 'orange (fruit)',
 'oyster',
 'pancake',
 'paneer_butter_masala',
 'pasta',
 'pastry',
 'peach',
 'pear',
 'pineapple',
 'pizza',
 'plain_white_rice',
 'pomegranate',
 'popcorn',
 'potato',


In [5]:
len(sorted(list(local_nutrition_db.keys())))

103

In [ ]:
['apple',
 'artichoke',
 'bagel',
 'balaleet',
 'bamya',
 'banana',
 'basbousa',
 'beer',
 'bell pepper',
 'bread',
 'broccoli',
 'burger',
 'burrito',
 'cabbage',
 'cake',
 'candy',
 'cantaloupe',
 'carrot',
 'cheese',
 'cocktail',
 'common fig',
 'cookie',
 'crab',
 'cream',
 'croissant',
 'cucumber',
 'dates_with_tahini',
 'fattoush',
 'fish',
 'french fries',
 'garden asparagus',
 'gers_ogaily',
 'grape',
 'grapefruit',
 'guacamole',
 'hamburger',
 'harees',
 'hot dog',
 'hummus',
 'ice cream',
 'jireesh',
 'kebab',
 'khabeesa',
 'kubba',
 'laban_drink',
 'labneh',
 'lemon',
 'lentil_soup',
 'lobster',
 'luqaimat',
 'majboos_dajaj',
 'mallooba(maqluba)',
 'mango',
 'modas_rice',
 'molokhia',
 'muffin',
 'muhammara',
 'murabyan',
 'musakhan_chicken',
 'mushroom',
 'om_ali',
 'orange (fruit)',
 'oyster',
 'pancake',
 'paneer_butter_masala',
 'pasta',
 'pastry',
 'peach',
 'pear',
 'pineapple',
 'pizza',
 'plain_white_rice',
 'pomegranate',
 'popcorn',
 'potato',
 'pretzel',
 'pumpkin',
 'radish',
 'rice_with_meat',
 'samosa',
 'sandwich',
 'shrimp',
 'squash (plant)',
 'strawberry',
 'submarine sandwich',
 'sushi',
 'tabouleh',
 'taco',
 'tahini',
 'tamarind_juice',
 'tamria(tamriyeh)',
 'tart',
 'tea_with_milk',
 'tomato',
 'waffle',
 'warak_enab',
 'watermelon',
 'wine',
 'winter melon',
 'zaatar',
 'zucchini']